### 18.2.3 Preprocessing Data With Pandas

### Questions for Data Preparation

Unsupervised learning doesn’t have a clear outcome or target variable like supervised learning, but it is used to find patterns. By properly preparing the data, we can select features that help us find patterns or groups.

Before we begin, consider these questions:

    -What knowledge do we hope to glean from running an unsupervised learning model on this dataset?
    -What data is available? What type? What is missing? What can be removed?
    -Is the data in a format that can be passed into an unsupervised learning model?
    -Can I quickly hand off this data for others to use?

Let’s address the first question on our list:

What knowledge do we hope to glean from running an unsupervised learning model on this dataset?

It’s a shopping dataset, so we can group together shoppers based on spending habits.

### Preprocessing - load data and review

In [2]:
# Import dependencies.
import pandas as pd
import csv
import os

In [3]:
# Load data.
file_path = os.path.join("Resources","shopping_data.csv")
print (file_path)

#Read iris.csv into a Pandas dataframe.
shopping_df = pd.read_csv(file_path, encoding="ISO-8859-1")
shopping_df.head(5)

Resources/shopping_data.csv


,CustomerID,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,Yes,19.0,15000,39.0
1,2,Yes,21.0,15000,81.0
2,3,No,20.0,16000,6.0
3,4,No,23.0,16000,77.0
4,5,No,31.0,17000,40.0


### 18.2.4 Data Selection 

Recall: in unsupervised learning, it's really important that you need to know how to select the data that will best help the model determine patterns or grouping. 

1. How to select data - What data is available? (CustomerID, Gender, Age, Annual Income, and Spending Score).

In [4]:
# Search columns in dataset to see what is listed
shopping_df.columns

Index(['CustomerID', 'Card Member', 'Age', 'Annual Income',
       'Spending Score (1-100)'],
      dtype='object')

-Now that we know what data we have, we can start thinking about possible analysis. For example, data points for features like Age and Annual Income might appear in our end result as groupings or clusters. However, there are no data points for items purchased, so our algorithms cannot discover related patterns.

In [5]:
### 2. What TYPE of data is available?
# Note: using the 'dtypes' method in Pandas will 
# show the DF data types.

# List dataframe data types
shopping_df.dtypes

CustomerID                  int64
Card Member                object
Age                       float64
Annual Income               int64
Spending Score (1-100)    float64
dtype: object

Which column doesn't contain a data type that we can use for our unsupervised learning model? 
A. CustomerID
B. Card Member
C. Annual Income
D. All columns contain correct data types

-B: card member displays as object, and unsupervised learning can only use numerical values (float or int)

### 3. What data is missing?

If any data is missing from the dataset, the following error will throw: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').


In [9]:
# Find null values
for column in shopping_df.columns:
    print(f"Column {column} has {shopping_df[column].isnull().sum()}null values")

Column CustomerID has 0null values
Column Card Member has 2null values
Column Age has 2null values
Column Annual Income has 0null values
Column Spending Score (1-100) has 1null values


Note: There will be a few rows with missing values that we’ll need to handle. The judgement call will be to either remove these rows or decide that the dataset is not suitable for our model. In this case, we’ll proceed with handling these values because they are a small percentage of the overall data.

When deciding to proceed, the percentage of data missing isn’t always the only determining factor. See the Note callout above for a resource on handling missing data.

### What data can be removed?
- Remove null (if necessary), duplicate data, and unusable data (i.e. non-numerical data)

In [11]:
# Drop null rows using dropna() method.
shopping_df = shopping_df.dropna()
shopping_df

,CustomerID,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,Yes,19.0,15000,39.0
1,2,Yes,21.0,15000,81.0
2,3,No,20.0,16000,6.0
3,4,No,23.0,16000,77.0
4,5,No,31.0,17000,40.0
...,...,...,...,...,...
198,199,No,35.0,120000,79.0
199,200,No,45.0,126000,28.0
200,201,Yes,32.0,126000,74.0
201,202,Yes,32.0,137000,18.0


In [12]:
# Drop duplicate entries- these can skew our results
# and don't tell us anything new about the data
# Find duplicate entries
print(f"Duplicate entries: {shopping_df.duplicated().sum()}")

Duplicate entries: 0


In [13]:
# Drop customerID column - doesn't tell us anything about customer shopping habits.
# Note: Card member although will be converted into binary later, 
# does show something about the data in regards to card holder status
# of the shopper.
shopping_df.drop(columns=["CustomerID"], inplace=True)
shopping_df.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,Yes,19.0,15000,39.0
1,Yes,21.0,15000,81.0
2,No,20.0,16000,6.0
3,No,23.0,16000,77.0
4,No,31.0,17000,40.0


### 18.2.5 Data Processing

The next step is to move on from what you (the user) want to get out of your data and on to what the unsupervised model needs out of the data.


Required for data processing:
- Null values are handled.

- Only numerical data is used.

- Values are scaled. In other words, data has been manipulated to ensure that the variance between the numbers won’t skew results.

In [14]:
# Convert string values in Card Member column to binary using elif statement.
def change_string(member):
    if member == "Yes":
        return 1
    else: 
        return 0

shopping_df["Card Member"] = shopping_df["Card Member"].apply(change_string)
shopping_df.head()
    

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15000,39.0
1,1,21.0,15000,81.0
2,0,20.0,16000,6.0
3,0,23.0,16000,77.0
4,0,31.0,17000,40.0


In [15]:
# Change scale on annual income (divide by 1000)
shopping_df["Annual Income"] = shopping_df["Annual Income"] / 1000
shopping_df.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


In [19]:
# Skill Drill: reformat the names of the columns to have no spaces
# or numbers 
shopping_drill_df = shopping_df.rename(columns={'Card Member':'Card_Member', 'Annual Income':'Annual_Income', 'Spending Score (1-100)':'Spending_Score'})
shopping_drill_df.head()                                              

,Card_Member,Age,Annual_Income,Spending_Score
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


In [ ]:
# Extra from Challenge assignment
# Use this for deleting rows that have boolean "True/false" values and
# all false values need to be dropped from the dataset.

#Example: sample_df = sample_df[sample_df.column3]
# original from stackoverflow: df = df[df.c3]


### 18.2.6 Data Transformation

-Transform data back into a user-friendly format (i.e. csv or Excel file; a readable format to be used in the future)


In [21]:
# Save/Transform cleaned data into a .csv file

# Set folder for output edited new_iris_data.csv file
output_file_path = os.path.join("Resources","shopping_data_cleaned.csv")

# Output edited shopping_DF to .csv file 
shopping_df.to_csv(output_file_path, index=False)
print (output_file_path)

Resources/shopping_data_cleaned.csv
